<a href="https://colab.research.google.com/github/seacatgm/-/blob/main/Classification/%D0%9F%D0%BE%D0%B8%D1%81%D0%BA%20%D0%BC%D0%BE%D1%88%D0%B5%D0%BD%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D1%85%20%D0%BE%D0%BF%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B9%20%D0%BF%D0%BE%20%D0%BA%D0%B0%D1%80%D1%82%D0%B5/AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import Model, Sequential # загружаем абстрактный класс базовой модели сети от кераса и последовательную модель
# Из кераса загружаем необходимые слои для нейросети
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, Conv2DTranspose, concatenate, Activation, MaxPooling2D, Conv2D, Conv1D, BatchNormalization, Dropout, LSTM, MaxPooling1D, Conv1DTranspose
from tensorflow.keras import backend as K # подтягиваем базовые керасовские функции
from tensorflow.keras.optimizers import Adam, Adamax, RMSprop # загружаем выбранный оптимизатор
from tensorflow.keras import utils # загружаем утилиты кераса
from tensorflow.keras.datasets import mnist, fashion_mnist # загружаем готовые базы mnist

import matplotlib.pyplot as plt # из библиотеки для визуализации данных возьмём интерфейс для построения графиков простых функций
from tensorflow.keras.preprocessing import image # модуль для отрисовки изображения
import numpy as np # библиотека для работы с массивами данных
import pandas as pd # библиотека для анализа и обработки данных
from PIL import Image # модуль для отрисовки изображения
from sklearn.model_selection import train_test_split # модуль для разбивки выборки на тренировочную/тестовую
from sklearn.preprocessing import StandardScaler # модуль для стандартизации данных

import os #
from google.colab import drive #модуль для работы с google диском 
import time

In [ ]:
drive.mount('/content/drive/') # монтируем google диск

Mounted at /content/drive/


In [ ]:
df = pd.read_csv("/content/drive/My Drive/neuro/base/creditcard.csv") # читаем базу

In [ ]:
df.head() # выводим первые пять строк

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
# Удаляем столбец со временем
data = df.drop(['Time'], axis=1)
# Нормализуем столбец с суммой транзакции
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))

frauds = data[data.Class == 1] # записываем мошеннические операции
normal = data[data.Class == 0] # записываем нормальные операции

In [ ]:
frauds.shape # смотрим какие размеры у мошеннических операций

(492, 30)

In [ ]:
normal.shape # смотрим какие размеры у нормальных операций

(284315, 30)

In [ ]:
RANDOM_SEED = 42 # задаем случайное значение

# берём нормальные операции и разбиваем на train_test(только 'X')
X_train, X_test = train_test_split(normal, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# Удаляем класс в обучающем наборе данных
#X_train = X_train.drop(['Class'], axis=1)
# Добавляем все мошеннические транзакции в тестовый набор данных
X_test = pd.concat([X_test, frauds])
# Делаем метки для тестового набора данных
y_test = X_test['Class']
# Удаляем класс у тестового набора данных
#X_test = X_test.drop(['Class'], axis=1)
# Преобразуем данные в массивы numpy
X_train = X_train.values
X_test = X_test.values


In [ ]:
print(X_train.shape) # взглянем на размеры обучающих данных

(227452, 30)


In [ ]:
def creditcardAutoencoder(): # объявляем автокодировщик для определения мошенич.транзакций
 # dataSize = xTrain.shape[1] # берём размеры X_train по второй оси(29)
  dataInput = Input(shape= (30, 1) ) # задаем эти размеры как входные в сеть
  
  x = Conv1D(30, 5, activation='relu', padding='same')(dataInput)
  x = BatchNormalization()(x)
  x = Conv1D(30, 5, activation='relu', padding='same')(x)
  x = MaxPooling1D(padding='same')(x)

  x = Conv1D(60, 5, activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv1D(60, 5, activation='relu', padding='same')(x)
  x = MaxPooling1D(pool_size=3,padding='same')(x)


  x = Conv1DTranspose(60,5,strides=1, padding='same', activation='relu')(x)
  x = BatchNormalization()(x)

  x = Conv1D(60, 5, activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv1D(60, 5, activation='relu', padding='same')(x)
  x = BatchNormalization()(x)

  x = Conv1DTranspose(30,5,strides=1, padding='same', activation='relu')(x)
  x = BatchNormalization()(x)

  x = Conv1D(30, 5, activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv1D(30, 5, activation='relu', padding='same')(x)
  x = BatchNormalization()(x)

  x = Flatten()(x)

  x = Dense(30, activation='linear')(x) # и через полносвязный слой размером 30 
  
  autoencoder = Model(inputs=dataInput, outputs=x) # собрали модель 
  
  autoencoder.compile(optimizer=RMSprop(lr=0.0001), loss='mse') # компилируем модель также, с выбором оптимайзера и среднеквадратичной ошибки
  
  return autoencoder 

In [ ]:
model = creditcardAutoencoder()
history = model.fit(X_test, X_test,
                    epochs=20,
                    batch_size=32) # загружаем данные в модель(X_train'ы) и обучаем

Epoch 1/20
1793/1793 [==============================] - 34s 17ms/step - loss: 1.2730
Epoch 2/20
1793/1793 [==============================] - 30s 17ms/step - loss: 0.3402
Epoch 3/20
1793/1793 [==============================] - 30s 17ms/step - loss: 0.2740
Epoch 4/20
1793/1793 [==============================] - 30s 17ms/step - loss: 0.1526
Epoch 5/20
1793/1793 [==============================] - 30s 17ms/step - loss: 0.1235
Epoch 6/20
1793/1793 [==============================] - 30s 17ms/step - loss: 0.1252
Epoch 7/20
1793/1793 [==============================] - 31s 17ms/step - loss: 0.0874
Epoch 8/20
1793/1793 [==============================] - 30s 17ms/step - loss: 0.0662
Epoch 9/20
1793/1793 [==============================] - 30s 17ms/step - loss: 0.0878
Epoch 10/20
1793/1793 [==============================] - 30s 17ms/step - loss: 0.0581
Epoch 11/20
1793/1793 [==============================] - 30s 17ms/step - loss: 0.0666
Epoch 12/20
1793/1793 [==============================] - 31s 17

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 30, 1)]           0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 30, 30)            180       
_________________________________________________________________
batch_normalization_24 (Batc (None, 30, 30)            120       
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 30, 30)            4530      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 15, 30)            0         
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 15, 60)            9060      
_________________________________________________________________
batch_normalization_25 (Batc (None, 15, 60)            240 

In [ ]:
def getAccByBias(bias): # функция будет принимать какое то пороговое значение
  isNormal = mse_normal < bias # если ошибка меньше порога - то транзакция нормальная
  isFrauds = mse_frauds > bias # если ошибка больше порога - то транзакция мошенническая

  accNormal = sum(isNormal) / len(isNormal) # вычисляем процент нормальных операций
  accFaruds = sum(isFrauds) / len(isFrauds) # вычисляем процент мошеннических операций
  sum_acc = accNormal + accFaruds
  #print("Распознано нормальных трансакций: ", round(100*accNormal), "%", sep="")
  #print("Распознано мошеннических трансакций: ", round(100*accFaruds), "%", sep="")
  #print("Средняя точность распознавания: ", round(50*(accNormal + accFaruds)), "%", sep="")
  return sum_acc

In [ ]:
def getAccByBias_percent(bias): # функция будет принимать какое то пороговое значение
  isNormal = mse_normal < bias # если ошибка меньше порога - то транзакция нормальная
  isFrauds = mse_frauds > bias # если ошибка больше порога - то транзакция мошенническая

  accNormal = sum(isNormal) / len(isNormal) # вычисляем процент нормальных операций
  accFaruds = sum(isFrauds) / len(isFrauds) # вычисляем процент мошеннических операций
  sum_acc = accNormal + accFaruds
  print("Распознано нормальных трансакций: ", round(100*accNormal), "%", sep="")
  print("Распознано мошеннических трансакций: ", round(100*accFaruds), "%", sep="")
  print("Средняя точность распознавания: ", round(50*(accNormal + accFaruds)), "%", sep="")

In [ ]:
predictions = model.predict(X_test) # делаем предсказание по X_test
mse = np.mean(np.power(X_test - predictions, 2), axis=1) # определяем среднеквадратичную ошибку по X_test и предсказанием по X_test
mse_normal = mse[y_test.values == 0] # среднеквадратичная ошибка на нормальных операциях
mse_frauds = mse[y_test.values == 1] # среднеквадратичная ошибка на мошеннических операциях

print("Минимальная ошибка нормальных трансакций:", round(min(mse_normal),4)) #найдем минимальную среднеквадратичную ошибку на нормальных операциях
print("Максимальная ошибка нормальных трансакций:", round(max(mse_normal),4)) #найдем максимальную среднеквадратичную ошибку на нормальных операциях
print("Средняя ошибка нормальных трансакций:", round(sum(mse_normal) / len(mse_normal),4)) # среднюю ошибку
print(' ')
print("Минимальная ошибка мошеннических трансакций:", round(min(mse_frauds),4)) #найдем минимальную среднеквадратичную ошибку на мошеннических операциях
print("Максимальная ошибка мошеннических трансакций:", round(max(mse_frauds),4)) #найдем максимальную среднеквадратичную ошибку на мошеннических операциях
print("Средняя ошибка мошеннических трансакций:", round(sum(mse_frauds) / len(mse_frauds),4)) # среднюю ошибку

Минимальная ошибка нормальных трансакций: 0.0058
Максимальная ошибка нормальных трансакций: 102.6194
Средняя ошибка нормальных трансакций: 0.0272
 
Минимальная ошибка мошеннических трансакций: 0.0366
Максимальная ошибка мошеннических трансакций: 21.5456
Средняя ошибка мошеннических трансакций: 0.6893


In [ ]:
mseFraud = round(sum(mse_frauds) / len(mse_frauds),4)
mseNorm = round(min(mse_normal),4)

In [ ]:
# Ищем лучший коэффициент для bias
best = []
number = 0
for i in np.linspace(0.001, 1, 100):
  if  getAccByBias(i)> 1.86:
    best.append(getAccByBias(i))
    print(round(getAccByBias(i),2), 'Сейчас такое, лучшее ------>',round(np.array(best).max(),2),   '         значение i = ', i)


1.87 Сейчас такое, лучшее ------> 1.87          значение i =  0.03127272727272727
1.92 Сейчас такое, лучшее ------> 1.92          значение i =  0.041363636363636366
1.9 Сейчас такое, лучшее ------> 1.92          значение i =  0.05145454545454546
1.86 Сейчас такое, лучшее ------> 1.92          значение i =  0.06154545454545455


In [ ]:
getAccByBias_percent(0.041363636363636366)

Распознано нормальных трансакций: 92%
Распознано мошеннических трансакций: 100%
Средняя точность распознавания: 96%
